## Git Token
  ##### Replace **your_personal_token** wtih git personal token.

### Fecth User data and save as users.csv

In [ ]:
import requests
import csv

# GitHub API base URL
BASE_URL = "https://api.github.com"

TOKEN = 'your_personal_token'  #token
CITY = 'Zurich'
MIN_FOLLOWERS = 50
users = []
HEADERS = {
        'Authorization': f'token {TOKEN}',
        'Accept': 'application/vnd.github.v3+json'
    }

def get_users_in_zurich():
    url = f"{BASE_URL}/search/users"
    query = f'location:{CITY} followers:>{MIN_FOLLOWERS}'
    page = 1
    while True:
      params = {'q': query, 'page': page, 'per_page': 100}  # Fetch up to 100 users per page
      response = requests.get(url, headers=HEADERS, params=params)

      if response.status_code == 200:
          data = response.json()
          users.extend(data['items'])
          if 'next' in response.links:  # Check if there is a next page
            page += 1
          else:
            break
      else:
          print("Error fetching users:", response.json())
          return []

def get_user_info(username):
    url = f"{BASE_URL}/users/{username}"
    response = requests.get(url, headers=HEADERS)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching info for {username}:", response.json())
        return None

def get_user_repos(username):
    url = f"{BASE_URL}/users/{username}/repos"
    response = requests.get(url, headers=HEADERS)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching repos for {username}:", response.json())
        return []

def save_to_csv(data, filename='users.csv'):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write the header
        writer.writerow([
            'login', 'name', 'company', 'location', 'email',
            'hireable', 'bio', 'public_repos', 'followers',
            'following', 'created_at'
        ])
        # Write the user data
        for user in data:
            writer.writerow(user)

def main():
    get_users_in_zurich()
    user_data = []

    for user in users:
        username = user['login']
        user_info = get_user_info(username)

        if user_info:
            company = user_info['company']
            if company:
              company = company.upper()

            repos = get_user_repos(username)
            public_repos_count = len(repos)
            created_at = user_info['created_at']
            email = user_info['email']

            user_data.append([
                user_info['login'],
                user_info['name'],
                company,
                user_info['location'],
                email,
                user_info['hireable'],
                user_info['bio'],
                public_repos_count,
                user_info['followers'],
                user_info['following'],
                created_at
            ])

    # Save the data to a CSV file
    save_to_csv(user_data)

if __name__ == "__main__":
    main()


### Fetch User repos data and save it as repositories.csv

In [ ]:
import requests
import pandas as pd

# GitHub API base URL
BASE_URL = "https://api.github.com"

# Define the headers for the API requests
HEADERS = {
    "Authorization": "token your_personal_token"  #personal access token
}

def get_user_repos(username):
    """Fetch the repositories for a given GitHub user."""
    url = f"{BASE_URL}/users/{username}/repos?sort=pushed&per_page=100"  # Changed to 100 for better pagination
    response = requests.get(url, headers=HEADERS)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching repos for {username}: {response.json()}")
        return []

def main():
    """Main function to load users from CSV, fetch their repositories, and save the data to another CSV."""
    try:
        # Load users from CSV
        users_df = pd.read_csv('users.csv')
    except FileNotFoundError:
        print("Error: users.csv file not found.")
        return

    repositories_data = []

    for _, user in users_df.iterrows():
        username = user['login']
        repos = get_user_repos(username)

        for repo in repos:
            repo_data = {
                'login': username,
                'full_name': repo['full_name'],
                'created_at': repo['created_at'],
                'stargazers_count': repo['stargazers_count'],
                'watchers_count': repo['watchers_count'],
                'language': repo['language'],
                'has_projects': repo['has_projects'],
                'has_wiki': repo['has_wiki'],
                'license_name': repo['license']['name'] if repo['license'] else 'No License'
            }
            repositories_data.append(repo_data)

    # Check if any repositories have been collected before saving
    if repositories_data:
        # Create a DataFrame and save to CSV
        repositories_df = pd.DataFrame(repositories_data)
        repositories_df.to_csv('repositories.csv', index=False)
        print("Repositories data saved to repositories.csv")
    else:
        print("No repositories found.")

if __name__ == "__main__":
    main()

Repositories data saved to repositories.csv


### To Authenticate the token

In [ ]:
!curl -H "Authorization: token your_personal_token" "https://api.github.com/user"

{
  "login": "itznoor998",
  "id": 159861560,
  "node_id": "U_kgDOCYdLOA",
  "avatar_url": "https://avatars.githubusercontent.com/u/159861560?v=4",
  "gravatar_id": "",
  "url": "https://api.github.com/users/itznoor998",
  "html_url": "https://github.com/itznoor998",
  "followers_url": "https://api.github.com/users/itznoor998/followers",
  "following_url": "https://api.github.com/users/itznoor998/following{/other_user}",
  "gists_url": "https://api.github.com/users/itznoor998/gists{/gist_id}",
  "starred_url": "https://api.github.com/users/itznoor998/starred{/owner}{/repo}",
  "subscriptions_url": "https://api.github.com/users/itznoor998/subscriptions",
  "organizations_url": "https://api.github.com/users/itznoor998/orgs",
  "repos_url": "https://api.github.com/users/itznoor998/repos",
  "events_url": "https://api.github.com/users/itznoor998/events{/privacy}",
  "received_events_url": "https://api.github.com/users/itznoor998/received_events",
  "type": "User",
  "user_view_type": "publ

In [ ]:
comment = "This code is for authenticate the token"
!curl --request GET \
--url "https://api.github.com/octocat" \
--header "Authorization: Bearer your_personal_token" \
--header "X-GitHub-Api-Version: 2022-11-28"


               MMM.           .MMM
               MMMMMMMMMMMMMMMMMMM
               MMMMMMMMMMMMMMMMMMM      ___________________________________
              MMMMMMMMMMMMMMMMMMMMM    |                                   |
             MMMMMMMMMMMMMMMMMMMMMMM   | Avoid administrative distraction. |
            MMMMMMMMMMMMMMMMMMMMMMMM   |_   _______________________________|
            MMMM::- -:::::::- -::MMMM    |/
             MM~:~ 00~:::::~ 00~:~MM
        .. MMMMM::.00:::+:::.00::MMMMM ..
              .MM::::: ._. :::::MM.
                 MMMM;:::::;MMMM
          -MM        MMMMMMM
          ^  M+     MMMMMMMMM
              MMMMMMM MM MM MM
                   MM MM MM MM
                   MM MM MM MM
                .~~MM~MM~MM~MM~~.
             ~~~~MM:~MM~~~MM~:MM~~~~
            ~~~~~~==~==~~~==~==~~~~~~
             ~~~~~~==~==~==~==~~~~~~
                 :~==~==~==~==~~


In [ ]:
!curl -H "Authorization: token your_personal_token" -i "https://api.github.com/rate_limit"

{
  "resources": {
    "core": {
      "limit": 5000,
      "used": 16,
      "remaining": 4984,
      "reset": 1730256311
    },
    "search": {
      "limit": 30,
      "used": 0,
      "remaining": 30,
      "reset": 1730254722
    },
    "graphql": {
      "limit": 5000,
      "used": 0,
      "remaining": 5000,
      "reset": 1730258262
    },
    "integration_manifest": {
      "limit": 5000,
      "used": 0,
      "remaining": 5000,
      "reset": 1730258262
    },
    "source_import": {
      "limit": 100,
      "used": 0,
      "remaining": 100,
      "reset": 1730254722
    },
    "code_scanning_upload": {
      "limit": 1000,
      "used": 0,
      "remaining": 1000,
      "reset": 1730258262
    },
    "actions_runner_registration": {
      "limit": 10000,
      "used": 0,
      "remaining": 10000,
      "reset": 1730258262
    },
    "scim": {
      "limit": 15000,
      "used": 0,
      "remaining": 15000,
      "reset": 1730258262
    },
    "dependency_snapshots": {
   

### Projects Questions' answers

#### Initialization

In [ ]:
import numpy as np, pandas as pd

In [ ]:
users_df = pd.read_csv('users.csv')

In [ ]:
users_df['login'][:5]

,login
0,IDouble
1,TheOfficialFloW
2,Seldaek
3,riscv
4,JonnyBurger


In [ ]:
users_df.columns

Index(['login', 'name', 'company', 'location', 'email', 'hireable', 'bio',
       'public_repos', 'followers', 'following', 'created_at', 'Unnamed: 11',
       'Unnamed: 12', 'raw_data_of_company_name'],
      dtype='object')

In [ ]:
users_df.shape

(475, 11)

In [ ]:
repo_df = pd.read_csv('repositories.csv')
repo_df.shape

(22696, 9)

In [ ]:
user = repo_df.login[len(repo_df)-1]
for i in range(users_df.shape[0]):
  if user==users_df.login[i]:
    print(i)
    break

474


#### Answers

##### OneShot ansers

In [ ]:
new_df = users_df.sort_values(by='followers',ascending=False)
result = list(new_df.login[:5])
for login in result:
  print(login,end=',') #answer1

IDouble,TheOfficialFloW,Seldaek,riscv,JonnyBurger,

In [ ]:
new_df = users_df.sort_values(by='created_at',ascending=True)
result = list(new_df.login[:5])
for login in result:
  print(login,end=',')#answer2

lejoe,uwolfer,matthiask,oscardelben,panterch,

In [ ]:
filtered_df = repo_df[repo_df['license_name'].notna()]
final = filtered_df.license_name.value_counts()[:6]
final.index[:3] #answer3

Index(['No License', 'MIT License', 'Apache License 2.0'], dtype='object', name='license_name')

In [ ]:
repo_df.license_name.unique()[:8]

array(['MIT License', 'No License', 'Apache License 2.0',
       'GNU Lesser General Public License v3.0', 'Other',
       'GNU General Public License v3.0', 'Open Software License 3.0',
       'BSD 3-Clause "New" or "Revised" License'], dtype=object)

In [ ]:
final = users_df.company.value_counts()
final.index[0]#answer4

'GOOGLE'

In [ ]:
filtered_df = repo_df[repo_df['language'].notna()]
final = filtered_df.language.value_counts()
final.index[0]#answer5

'Python'

In [ ]:
repo_df['created_at'] = pd.to_datetime(repo_df.created_at)

In [ ]:
repo_df.created_at[:1].dtype

datetime64[ns, UTC]

In [ ]:
filtered_df = repo_df[repo_df['language'].notna()]
date_str = '2020/12/31'
min_date = pd.to_datetime(date_str).tz_localize('UTC')
condition = filtered_df.created_at >= min_date
ans = filtered_df.loc[condition].language.value_counts()
ans.index[2] # answer6

'JavaScript'

In [ ]:
ans[:4]

,count
language,
Python,1011
TypeScript,438
JavaScript,429
C++,259


In [ ]:
import pandas as pd

# Original date string
date_str = '2020/12/31'

# Convert to datetime64 in UTC
date_time = pd.to_datetime(date_str).tz_localize('UTC')

print(date_time)


2020-12-31 00:00:00+00:00


In [ ]:
average_stars = repo_df.groupby('language').stargazers_count.mean()
ans = average_stars.sort_values(ascending=False)
ans.index[0]#answer7

'BitBake'

In [ ]:
copy_df = users_df
copy_df['leaders_strenth'] = copy_df.followers/(copy_df.following+1)
new_df = copy_df.sort_values(by='leaders_strenth',ascending=False)
result = new_df.login[:5]
for login in result:
  print(login,end=',') #answer8

riscv,bpasero,Seldaek,egamma,ethz-asl,

In [ ]:
ans = users_df.public_repos.corr(users_df.followers)
round(ans,3) #answer9

0.068

In [ ]:
X = users_df.public_repos.values
X = X.reshape(len(X),1)
y = users_df.followers.values
y = y.reshape(len(y),1)
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X,y)
slope = lr.coef_[0][0]
round(slope,3) #answer10

12.897

In [ ]:
ans = repo_df.has_wiki.corr(repo_df.has_projects)
round(ans,3) #answer11

0.336

In [ ]:
hireable = users_df.loc[users_df.hireable==True]
rest_hireable = users_df.loc[users_df.hireable !=True]
hire_yes = hireable.following.mean()
hire_rest = rest_hireable.following.mean()
result = hire_yes - hire_rest
round(result,3) #answer12

-834.073

In [ ]:
bios = [ ]
new_df = users_df.dropna(subset='bio')
for bio in list(new_df['bio']):
  length = len(str(bio).split())
  bios.append(length)
bios = np.array(bios).reshape(len(bios),1)

In [ ]:
followers = new_df.followers.values
followers = followers.reshape(len(followers),1)
from sklearn.linear_model import LinearRegression
lg = LinearRegression()
lg.fit(bios,followers)
ans = lg.coef_[0][0]
round(ans,3) #answer13

40.527

##### 14. Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated
 * Users login

In [ ]:
from datetime import datetime
dates = repo_df.created_at.values
weekend = [ ]
for date in dates:
  dt = datetime.strptime(date,'%Y-%m-%dT%H:%M:%SZ')
  if dt.weekday() >= 5:
    weekend.append(True)
  else:
    weekend.append(False)
weekend = np.array(weekend)

In [ ]:
repo_df['weekend'] = weekend;

In [ ]:
condition14 = repo_df.weekend==True
new_df = repo_df.loc[condition14]
data = new_df.groupby('login').count().sort_values(by='created_at',ascending=False)[:5]
result = list(data.index)
for login in result:
  print(login,end=',') #answer14

kynan,lris,yati-sagade,Sadullah-TANRIKULU,shuhei,

##### 15. Do people who are hireable share their email addresses more often?
* [fraction of users with email when hireable=true] minus [fraction of users with email for the rest] (to 3 decimal places, e.g. 0.123 or -0.123)

In [ ]:
new_df1 = users_df.loc[(users_df['email'].isna()==False) & (users_df.hireable==True)]
hireable_email = new_df1.shape[0]

new_df2 = users_df.loc[(users_df['email'].isna()==False) & (users_df.hireable!=True)]
rest_email = new_df2.shape[0]

size1 = users_df.loc[(users_df.hireable==True)].shape[0]
size2 = users_df.loc[(users_df.hireable!=True)].shape[0]

In [ ]:
frac1 = hireable_email/size1
frac2 = rest_email/size2
round(frac1-frac2,3)

0.066

##### 16. Let's assume that the last word in a user's name is their surname (ignore missing names, trim and split by whitespace.) What's the most common surname? (If there's a tie, list them all, comma-separated, alphabetically)
* Most common surname(s)

In [ ]:
new_df = users_df.dropna(subset='name')
surnames = [ ]
for name in list(new_df.name.values):
  out = name.split(' ')
  surnames.append(out[-1])
surnames = np.array(surnames)
new_df['surname'] = surnames;

<ipython-input-84-13c4a56cbbc8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['surname'] = surnames;


In [ ]:
result = new_df.groupby('surname').count().sort_values(by='login',ascending=False)
result[:5]

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at,Unnamed: 11,Unnamed: 12,raw_data_of_company_name
surname,,,,,,,,,,,,,,
Wang,4,4,3,4,2,1,3,4,4,4,4,0,0,3
Li,4,4,2,4,3,0,4,4,4,4,4,0,0,2
Lab,3,3,0,3,0,0,1,3,3,3,3,0,0,0
Group,2,2,0,2,0,0,1,2,2,2,2,0,0,0
Smith,2,2,1,2,2,0,0,2,2,2,2,0,0,1


In [ ]:
final = list(result.index[:2])
final.sort()
for sname in final:
  print(sname,end=',')

Li,Wang,

## Project's Inferences

In [ ]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
plt.style = 'seaborn'

In [ ]:
user_df = pd.read_csv('users.csv')
repo_df = pd.read_csv('repositories.csv')

In [ ]:
df = pd.merge(user_df,repo_df,on='login',how='inner')

### Data Analysis

#### has_projects and followers

In [ ]:
X = df.has_projects.values
X = X.reshape(len(X),1)
y = df.followers.values
y = y.reshape(len(y),1)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X,y)
lr.coef_[0][0]

88.56665062137796

#### For stargazers_count and has_projects

In [ ]:
X2 = repo_df.has_projects.values
X2 = X2.reshape(len(X2),1)

y2 = repo_df.stargazers_count.values
y2 = y2.reshape(len(y2),1)

In [ ]:
from sklearn.linear_model import LinearRegression
lr2 = LinearRegression()
lr2.fit(X2,y2)
lr2.coef_[0][0]

-130.99936471415313

### Inference

In [ ]:
'''
Normally, if a repo has projects, it is more likely to get more star.
But I saw a negetive trend in a repo-data
has_projects and stargazers_count
'''

### Rough

In [ ]:
new = df.loc[df['has_projects']==True].sort_values(by='followers',ascending=False)
new.groupby('login').followers.sum()

,followers
login,
0xB10C,24120
0xabu,420
19reborn,1955
AIWintermuteAI,12328
ANYbotics,42496
...,...
zhendongsu,188
zhengyuf,632
zimrick,650
